In [ ]:
#################### Importing Libraries ##############
import numpy as np
import random
import sys, os
import json
import argparse
import io  
import tflearn
import tensorflow as tf
import time
from tensorflow.contrib.rnn import LSTMCell
from gensim.models import FastText as fText
from nltk.corpus import stopwords
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
###################get data
datasets = {}
for fname in ['train', 'dev', 'test']:
    data = []
    for line in open('%s/%s.res' % (dataset, fname)):
        s = json.loads(line.strip())
        if len(s) > 0:
            data.append(s)
    datasets[fname] = data

    global_subwords = {}
PREmodel = fText.load_fasttext_format('wiki.en.bin')
fastTextVocab = PREmodel.wv.vocab
print("Vocabulary built\n")
stop_words = set(stopwords.words('english'))  

In [ ]:
# Get all substrings of string 
def all_substr(word):
    res = [word[i: j] for i in range(len(word)) for j in range(i + 1, len(word) + 1)] 
    return res

def all_subwords(word):            
    subword_list = all_substr(word)
    subWInDict = []
    for i in subword_list:
        if i in fastTextVocab and len(i)>3:
            if not i in stop_words:
                subWInDict.append(i)
    return subWInDict

wordcount = {}

def dfs(node):
    if node.__contains__('children'):
        dfs(node['children'][0])
        dfs(node['children'][1])
    else:
        word = node['word'].lower()
        wordcount[word] = wordcount.get(word, 0) + 1
    
for fname in ['train', 'dev', 'test']:
    for sent in self.origin[fname]:
        dfs(sent)
for ind, ite in enumerate(list(wordcount)):
    subW = []
    if len(ite)>4:
        subW = all_subwords(ite)
        for w in subW:
            wordcount[w] = wordcount.get(w, 0) + 1
    if len(subW)==0:
        global_subwords[ite] = [ite]
    else:
        global_subwords[ite] = subW
words = wordcount.items()
sorted(words,key = lambda x : x[1], reverse = True)
wordlist = {item[0]: index+1 for index, item in enumerate(words)}

In [ ]:
############ Word Vectors
dim = 300
wordvector = []
similarcnt = 0
print("Creating word vectors\n")
for i,x in enumerate(wordlist):
    if x in fastTextVocab:
        wordvector.append(np.array(PREmodel.wv[x]))
    else:
        similarcnt += 1
        wordvector.append(np.array(PREmodel.wv[PREmodel.wv.most_similar(x)[0][0]]))
wordvector = np.array(wordvector, dtype=np.float32)
print (similarcnt, "similar words used in wordvector")
print (len(wordvector), "words in total")

In [ ]:
data = {}
global_max = 0
for fname in ['train', 'dev', 'test']:
    data[fname] = []
    for node in origin[fname]:
        words = []
        if node.__contains__('children'):
            dfs(node['children'][0], words)
            dfs(node['children'][1], words)
        else:
            subw = global_subwords[node['word'].lower()]
            for wor in subw:
                words.append(self.wordlist[wor])
        lens = len(words)
        if lens > global_max:
            global_max = lens
        if maxlength < lens:
            print ("******Alert****", lens)
        words += [0] * (maxlength - lens)
        soultion = np.zeros(grained, dtype=np.float32)
        solution[int(sent['rating'])] += 1.0
        now = {'words': np.array(words), \
                'answer': solution,\
                'length': lens}
        data[fname].append(now)
print("Global Max: ", global_max )
train_data, dev_data, test_data = data['train'], data['dev'], data['test']

In [ ]:
argv = sys.argv[1:]
parser = argparse.ArgumentParser()
parser.add_argument('--fasttest', type=int, default=0)
parser.add_argument('--seed', type=int, default=int(1000*time.time()))
parser.add_argument('--word_vector', type=str, default='../glove-Copy1.6B.300d.txt')
parser.add_argument('--dim', type=int, default=300)
parser.add_argument('--tau', type=float, default=0.1)
parser.add_argument('--dropout', type=float, default=0.5)
parser.add_argument('--sample_cnt', type=int ,default=5)
parser.add_argument('--LSTMpretrain', type=str, default='')
parser.add_argument('--RLpretrain', type=str, default='')
parser.add_argument('--maxlength', type=int, default=256)
parser.add_argument('--grained', type=int, default=4)
parser.add_argument('--optimizer', type=str, default='Adam')
parser.add_argument('--alpha', type=float, default=0.1)
parser.add_argument('--epsilon', type=float, default=0.05)
parser.add_argument('--dataset', type=str, default='../TrainData/AG')
parser.add_argument('--lr', type=float, default=0.0005)
parser.add_argument('--epoch', type=int, default=3)
parser.add_argument('--batchsize', type=int, default=64)
args, _ = parser.parse_known_args(argv)
random.seed(args.seed)

In [ ]:
if args.fasttest == 1:
    train_data = train_data[:5000]
    dev_data = dev_data[:1000]
    test_data = test_data[:1000]

print ("train_data ", len(train_data))
print ("dev_data", len(dev_data))
print ("test_data", len(test_data))

In [ ]:
#previous epoch loss
prevLoss = [0.]*len(train_data)
print("PrevLoss length, Type, List element type: ", len(prevLoss), type(prevLoss), type(prevLoss[0]))
def train(sess, actor, critic, train_data, batchsize, sample=5, LSTM=True, RL=True):
    print ("training : total ", len(train_data))
    for b in range(int(len(train_data) / batchsize)):
        datas = train_data[b * batchsize: (b+1) * batchsize]
        totalloss = 0.
        critic.assign_active_network()
        actor.assign_active_network()
        for j in range(batchsize):
            data = datas[j]
            inputs, answer, length = data['words'], data['answer'], data['length']
            if RL:
                actionlist, statelist, losslist = [], [], []
                averageloss = 0.
                tempLoss = 0.
                for i in range(sample):
                    vec = critic.wordvector_find([inputs])
                    current_lower_state = np.zeros((1, 2*args.dim), dtype=np.float32)
                    actions = []
                    states = []
                    for pos in range(length):
                        predicted = actor.predict_target(current_lower_state, [vec[0][pos]])
                        states.append([current_lower_state, [vec[0][pos]]])
                        if Random:
                            if random.random() > args.epsilon:
                                action = (0 if random.random() < predicted[0] else 1)
                            else:
                                action = (1 if random.random() < predicted[0] else 0)
                        else:
                            action = np.argmax(predicted)
                        actions.append(action)
                        if action == 1:
                            out_d, current_lower_state = critic.lower_LSTM_target(current_lower_state, [[inputs[pos]]])

                    Rinput = []
                    for (i, a) in enumerate(actions):
                        if a == 1:
                            Rinput.append(inputs[i])
                    Rlength = len(Rinput)
                    if Rlength == 0:
                        actions[length-2] = 1
                        Rinput.append(inputs[length-2])
                        Rlength = 1
                    Rinput += [0] * (args.maxlength - Rlength)
                
                    statelist.append(states)
                    actionlist.append(actions)
                    out, loss = critic.getloss([Rinput], [Rlength], [answer])
                    loss += (float(Rlength) / length) **2 *0.15
                    averageloss += loss
                    losslist.append(loss)
                
                averageloss /= sample              
                tempLoss = averageloss - prevLoss[b * batchsize + j] 
                prevLoss[b * batchsize + j] = averageloss - (float(Rlength) / length)                    
                aveloss = tempLoss 
                totalloss += averageloss
                grad = None
                if LSTM:
                    out, loss, _ = critic.train([Rinput], [Rlength], [solution])
                for i in range(sample):
                    for pos in range(len(actionlist[i])):
                        rr = [0., 0.]
                        rr[actionlist[i][pos]] = (losslist[i] - averageloss) * args.alpha
                        g = actor.get_gradient(statelist[i][pos][0], statelist[i][pos][1], rr)
                        if grad == None:
                            grad = g
                        else:
                            grad[0] += g[0]
                            grad[1] += g[1]
                            grad[2] += g[2]
                actor.train(grad)
                else:
                    out, loss, _ = critic.train([inputs], [length], [answer])
                    totalloss += loss
        if RL:
            actor.update_target_network()
            if LSTM:
                critic.update_target_network()
        else:
            critic.assign_target_network()
        if (b + 1) % 200 == 0:
            acc_test = test(sess, actor, critic, test_data, noRL= not RL)
            acc_dev = test(sess, actor, critic, dev_data, noRL= not RL)
            print ("batch ",b , "loss ", totalloss, "test acc: ", acc_test, ", dev acc: ", acc_dev)

In [ ]:
configure = tf.ConfigProto()
configure.gpu_options.allow_growth = True
with tf.Session(configure = configure) as sess:
    critic = LSTM_CriticNetwork(sess, args.dim, args.optimizer, args.lr, args.tau, args.grained, args.maxlength, args.dropout, word_vector) 
    actor = ActorNetwork(sess, args.dim, args.optimizer, args.lr, args.tau)
    for i in range(0, 3):
        train(sess, actor, critic, train_data, args.batchsize, args.sample_cnt, LSTM_trainable=False)
        acc_test = test(sess, actor, critic, test_data)
        acc_dev = test(sess, actor, critic, dev_data)
        print ("RL ", i, "test: ", acc_test, ", dev: ", acc_dev)
    print ("RL Model pretrained")

    sess.run(tf.global_variables_initializer())
    for i in range(0, 3):
        train(sess, actor, critic, train_data, args.batchsize, args.sample_cnt, RL_trainable=False)
        critic.assign_target_network()
        acc_test = test(sess, actor, critic, test_data, True)
        acc_dev = test(sess, actor, critic, dev_data, True)
        print ("LSTM ",i, "test: ", acc_test, ", dev: ", acc_dev)
    print ("LSTM Model pretrained")
    
    for e in range(args.epoch):
        train(sess, actor, critic, train_data, args.batchsize, args.sample_cnt)
        acc_test = test(sess, actor, critic, test_data)
        acc_dev = test(sess, actor, critic, dev_data)
        print ("epoch ", e, "test acc: ", acc_test, ", dev acc: ", acc_dev)

In [ ]:
class Actor(object):
    def __init__(self, sess, dim, optimizer, learning_rate, tau):
        self.global_step = tf.Variable(0, trainable=False, name="Actor")
        self.sess = sess
        self.dim = dim
        self.learning_rate = tf.train.exponential_decay(learning_rate, self.global_step, 1000, 0.95, staircase=True)
        self.tau = tau
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.input_l, self.input_d, self.scaled_out = self.create_actor_network()
        self.network_params = tf.trainable_variables()[self.num_other_variables:]
        self.target_input_l, self.target_input_d, self.target_scaled_out = self.create_actor_network()
        self.target_network_params = tf.trainable_variables()[self.num_other_variables + len(self.network_params):]

        self.update_target_network_params = [self.target_network_params[i].assign(
            tf.multiply(self.network_params[i], self.tau) + tf.multiply(self.target_network_params[i], 
            1 - self.tau)) for i in range(len(self.target_network_params))]        
        self.assign_active_network_params = [self.network_params[i].assign(self.target_network_params[i]) 
                                             for i in range(len(self.network_params))]
        self.action_gradient = tf.placeholder(tf.float32, [2])
        self.log_target_scaled_out = tf.log(self.target_scaled_out)
        self.actor_gradients = tf.gradients(self.log_target_scaled_out, self.target_network_params,
                                            self.action_gradient)
        self.gradients = [tf.placeholder(tf.float32, [600,1]), 
                        tf.placeholder(tf.float32, [1,]),
                        tf.placeholder(tf.float32, [300, 1])]
        self.optimize = self.optimizer.apply_gradients(zip(self.gradients, self.network_params[:-1]), 
                                                       global_step=self.global_step)

    def create_actor_network(self):
        input_l = tf.placeholder(tf.float32, shape=[1, self.dim*2])
        input_d = tf.placeholder(tf.float32, shape=[1, self.dim])        
        t1 = tflearn.fully_connected(input_l, 1)
        t2 = tflearn.fully_connected(input_d, 1)
        scaled_out = tflearn.activation(tf.matmul(input_l,t1.W) + tf.matmul(input_d,t2.W) + t1.b, activation = 'sigmoid')        
        s_out = tf.clip_by_value(scaled_out[0][0], 3e-4, 1 - 3e-4)
        scaled_out = tf.stack([1.0 - s_out, s_out])
        return input_l, input_d, scaled_out

In [ ]:
class LSTM_Critic(object):
    def __init__(self, sess, dim, optimizer, learning_rate, tau, grained, max_length, dropout, wordvector):
        self.global_step = tf.Variable(0, trainable=False, name="LSTM")
        self.sess = sess
        self.max_length = max_length
        self.dim = dim
        self.learning_rate = tf.train.exponential_decay(learning_rate, self.global_step, 1000, 0.95, staircase=True)
        self.tau = tau
        self.grained = grained
        self.dropout = dropout
        self.init = tf.random_uniform_initializer(-0.01, 0.01, dtype=tf.float32)
        self.L2regular = 0.0001
        print ("optimizer: ", optimizer)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.keep_prob = tf.placeholder(tf.float32, name="keepprob")
        self.num_other_variables = len(tf.trainable_variables())
        self.wordvector = tf.get_variable('wordvector', dtype=tf.float32, initializer=wordvector, trainable=True)

        self.lower_cell_state, self.lower_cell_input, self.lower_cell_output, self.lower_cell_state1 = self.LSTM_cell('Lower/Active')

        self.inputs, self.length, self.out = self.create_critic_network("Active")
        self.network_params = tf.trainable_variables()[self.num_other_variables:]        
        self.target_wordvector = tf.get_variable('wordvector_target', dtype=tf.float32, initializer=wordvector, trainable=True)

        self.target_lower_cell_state, self.target_lower_cell_input, self.target_lower_cell_output, self.target_lower_cell_state1 = self.LSTM_cell('Lower/Target')
        
        self.target_inputs, self.target_length, self.target_out = self.create_critic_network("Target")
        self.target_network_params = tf.trainable_variables()[len(self.network_params)+self.num_other_variables:]
        self.update_target_network_params = \
                [self.target_network_params[i].assign(\
                tf.multiply(self.network_params[i], self.tau)+\
                tf.multiply(self.target_network_params[i], 1 - self.tau))\
                for i in range(len(self.target_network_params))]
        
        self.assign_target_network_params = \
                [self.target_network_params[i].assign(\
                self.network_params[i]) for i in range(len(self.target_network_params))]
        self.assign_active_network_params = \
                [self.network_params[i].assign(\
                self.target_network_params[i]) for i in range(len(self.network_params))]

        self.ground_truth = tf.placeholder(tf.float32, [1,self.grained], name="ground_truth")        
        
        self.loss_target = tf.nn.softmax_cross_entropy_with_logits(labels=self.ground_truth, logits=self.target_out)
        self.loss = tf.nn.softmax_cross_entropy_with_logits(labels=self.ground_truth, logits=self.out)
        self.loss2 = 0
        with tf.variable_scope("Lower/Active", reuse=True):
            self.loss2+= tf.nn.l2_loss(tf.get_variable('lstm_cell/kernel'))
        with tf.variable_scope("Active/pred", reuse=True):
            self.loss2+= tf.nn.l2_loss(tf.get_variable('W'))
        self.loss += self.loss2 * self.L2regular
        self.loss_target += self.loss2 * self.L2regular
        self.gradients = tf.gradients(self.loss_target, self.target_network_params)
        self.optimize = self.optimizer.apply_gradients(zip(self.gradients, self.network_params), global_step = self.global_step)
        
        self.num_trainable_vars = len(self.network_params) + len(self.target_network_params)
        self.WVinput, self.WVvec = self.create_wordvector_find()

    def create_critic_network(self, Scope):
        inputs = tf.placeholder(shape=[1, self.max_length], dtype=tf.int32, name="inputs")
        length = tf.placeholder(shape=[1], dtype=tf.int32, name="length")
       
        if Scope[-1] == 'e':
            vec = tf.nn.embedding_lookup(self.wordvector, inputs)
        else:
            vec = tf.nn.embedding_lookup(self.target_wordvector, inputs)
        cell = LSTMCell(self.dim, initializer=self.init, state_is_tuple=False)
        
        with tf.variable_scope("Lower", reuse=True):
            out, _ = tf.nn.dynamic_rnn(cell, vec, length, dtype=tf.float32, scope=Scope)
        out = tf.gather(out[0], length-1)
        
        out = tflearn.dropout(out, self.keep_prob)
        out = tflearn.fully_connected(out, self.grained, scope=Scope+"/pred", name="pred")
        return inputs, length, out
    
    def LSTM_cell(self,Scope):
        cell = LSTMCell(self.dim, initializer=self.init, state_is_tuple=False)
        state = tf.placeholder(tf.float32, shape = [1, cell.state_size], name="state")
        inputs = tf.placeholder(tf.int32, shape = [1, 1], name="input")
        if Scope[-1] == 'e':
            vec = tf.nn.embedding_lookup(self.wordvector, inputs)
        else:
            vec = tf.nn.embedding_lookup(self.target_wordvector, inputs)
        with tf.variable_scope(Scope, reuse=False):
            out, state1 = cell(vec[:,0,:], state)
        return state, inputs, out, state1

    def getloss(self, inputs, length, ground_truth):
        return self.sess.run([self.target_out, self.loss_target], feed_dict={
            self.target_inputs: inputs,
            self.target_length: length,
            self.ground_truth: ground_truth,
            self.keep_prob: 1.0})

    def train(self, inputs, length, ground_truth):
        return self.sess.run([self.target_out, self.loss_target, self.optimize], feed_dict={
            self.target_inputs: inputs,
            self.target_length: length,
            self.ground_truth: ground_truth,
            self.keep_prob: self.dropout})